## Modules over the ring of integers $\mathbb Z$

In [14]:
from collections import Counter

class int_comb(Counter):
    '''...'''
    
    def __init__(*args, **kwds):
        '''...'''
        self, *args = args
        super(int_comb, self).__init__(*args, **kwds)
        
        if not all( [type(v) is int for v in self.values()] ):
            raise TypeError('values must be integers') 
    
    def __add__(self, other):
        '''...'''
        answer = int_comb(self)
        answer.update(other)
        return int_comb( {k:v for k,v in answer.items() if v} )
    
    def __sub__(self, other):
        '''...'''
        answer = int_comb(self)
        answer.subtract(other)
        return int_comb( {k:-v for k,v in answer.items() if v} )
    
    def __mod__(self, p):
        '''...'''
        return int_comb( {k:v%p for k,v in self.items() if v % p} )
    
    def __neg__(self):
        '''...'''
        return int_comb( {k:-v for k,v in self.items() if v} )
    
    def scalar_mul(self, c):
        '''...'''
        for key, value in self.items():
            self[key] = value*c
            
        return self
    
    def __iadd__(self, other):
        '''...'''
        self.update(other)
        self.remove_zeros()
        return self
    
    def __isub__(self, other):
        '''...'''
        self.subtract(other)
        self.remove_zeros()
        return self
    
    def __imod__(self, p):
        '''...'''
        for key, value in self.items():
            self[key] = value % p
            
        return self
    
    def __str__(self):
        '''...'''
        self.remove_zeros()
        if not self:
            return '0'
        else:
            answer = '' 
            for key, value in self.items():
                if value < 0:
                    answer += f'{value}{key}'
                elif value == 1:
                    answer += f'+{key}'
                elif value > 1:
                    answer += f'+{value}{key}'    
            if answer[0] == '+':
                answer = answer[1:]

            return answer
        
    def remove_zeros(self):
        '''In place removal of keys with 0 value'''
        zeros = [k for k,v in self.items() if not v]
        for key in zeros:
            del self[key]    

## Modules over the ring $\mathbb Z/ p\mathbb Z$

In [3]:

#_________________________________79_characters________________________________
class mod_p_comb(int_comb):
    '''...'''
    
    prime = 3
    
    def __init__(*args, **kwds):
        '''...'''
        self, *args = args
        super(mod_p_comb, self).__init__(*args, **kwds)
        self.reduce_values()
        
    def reduce_values(self):
        '''In place mod p reduction of the values'''
        super().__imod__(self.prime)
        super().remove_zeros()
        
    def __add__(self, other):
        '''...'''
        if self.prime != other.prime:
            raise ValueError('same prime for both')
            
        return mod_p_comb( super().__add__(other) )
    
    def __sub__(self, other):
        '''...'''
        if self.prime != other.prime:
            raise ValueError('same prime for both')
            
        return mod_p_comb( super().__sub__(other) )
    
    def __neg__(self):
        '''...'''
        return mod_p_comb( {k:-v for k,v in self.items() if v} )
    
    def scalar_mul(self, c):
        '''...'''
        super().scalar_mul(c)
        self.reduce_values()

## ???

In [13]:
#_________________________________79_characters________________________________
class cyclic_mod_p_alg_elmt(mod_p_comb):
    '''...'''
    
    def __init__(*args, **kwds):
        '''...'''
        self, *args = args
        super(cyclic_mod_p_alg_elmt, self).__init__(*args, **kwds)
        if not all([isinstance(k,int) for k in self.keys()]):
            raise TypeError('keys must be integers')
        self.reduce_keys()
    
    def reduce_keys(self):
        '''in place mod p reduction of the keys'''
        aux = list(self.items())
        self.clear()
        for k,v in aux:
            self[k%self.prime] += v
            
    def __mul__(self, other):
        '''...'''
        if self.prime != other.prime:
            raise ValueError('same prime for both')
        answer = cyclic_mod_p_alg_elmt()
        for k1,v1 in self.items():
            for k2,v2 in other.items():
                answer[k1+k2] += v1*v2
        answer.reduce_keys()
        super(cyclic_mod_p_alg_elmt, answer).reduce_values()
        
        return answer
        
    def __call__(self, other):
        '''...'''
        if isinstance(other, cyclic_mod_p_alg_elmt):
            return self*other
#         if isinstance(other, CyclicBarElmt):
#             assert self.prime == other.prime, \
#               'both elements must have the same prime'
#             answer = Counter()
#             for k,v1 in self.items():
#                 for x,v2 in other.items():
#                     y = tuple(k+i % self.prime for i in x)
#                     answer[y] += v1*v2
#             return CyclicBarElmt(answer, self.prime)
    
    def __repr__(self):
        '''...'''
        s = super().__repr__()
        if self:
            return s.replace('})',f'}}, p={self.prime})')
        if not self:
            return s.replace('()', f'({{}}, p={self.prime})')
    
    def __str__(self):
        '''...'''
        self.remove_zeros()
        if not self:
            return '0'
        else:
            answer = '' 
            for key, value in self.items():
                if value < 0 and key:
                    answer += f'{value}a^{key}'
                elif value < 0 and not key:
                    answer += f'{value}'
                elif value == 1 and key:
                    answer += f'+a^{key}'
                elif value == 1 and not key:
                    answer += f'+1'
                elif value > 1 and key:
                    answer += f'+{value}a^{key}'
                elif value > 1 and not key:
                    answer += f'+{value}'
            if answer[0] == '+':
                answer = answer[1:]

            return answer.replace('^1','')
        
    @staticmethod
    def norm_elmt():
        '''...'''
        elmt = {i:1 for i in range(mod_p_comb.prime)}
        return cyclic_mod_p_alg_elmt(elmt)
    
    @staticmethod
    def transpo_elmt():
        '''...'''
        elmt = {1:1, 0:-1}
        return cyclic_mod_p_alg_elmt(elmt)
    
    @staticmethod
    def all_elmts():
        '''...'''
        p = mod_p_comb.prime
        return ( cyclic_mod_p_alg_elmt( dict(zip(range(p), coeffs)) ) 
                 for coeffs in product(range(p),repeat=p) )

## ???

In [12]:
# TODO: write a class modeling bar complexes as a subclass 
# IntegralCombination having a boundary method and a remove 
# degenerate simplices method 
#_________________________________79_characters________________________________
class BarElmt(int_comb):
    def __init__(self, counter):
        '''...'''
        super().__init__()
        assert all([type(x) is tuple for x in self.keys()]),\
                    'keys must be tuples'
        
    def boundary(self):
        bdry = BarElmt()
        sign = {0:1, 1:-1}
        for spx, coeff in self.items():
            for i in range(len(spx)):
                bdry += BarElmt({tuple(spx[:i]+spx[i+1:])
                                 :sign[i%2]*coeff})
        return bdry
    
#_________________________________79_characters________________________________

## ???

In [15]:
# TODO: review this class
#_________________________________79_characters________________________________
class CyclicBarElmt(mod_p_comb):
    def __init__(self, element, p=3):
        '''...'''
        super().__init__(element)
        self.prime = p
        assert all([type(x) is tuple and 
                    all(type(i) is int for i in x) 
                    for x in self.keys()]), \
                    'keys must be tuples of integers'
        self.reduce_keys()
        self.reduce_values()
        
    def reduce_keys(self):
        '''In place mod p reduction of the keys'''
        aux = list(self.items())
        self.clear()
        for x,v in aux:
            y = tuple(i%self.prime for i in x)
            self[y] += v
        self %= self.prime
    
    def reduce_values(self):
        '''In place mod p reduction of the values'''
        self %= self.prime
        self.remove_zeros()
        
    def __add__(self, other):
        '''...'''
        assert self.prime == other.prime, \
          'both elements must have the same prime'
        answer = super().__add__(other)
        return CyclicBarElmt(answer, self.prime)
    
    def __sub__(self, other):
        '''...'''
        assert self.prime == other.prime, \
          'both elements must have the same prime'
        answer = super().__sub__(other)
        return CyclicBarElmt(answer, self.prime)
    
    def scalar_mul(self, c):
        '''...'''
        super().scalar_mul(c)
        self.reduce_values()
        
    def __repr__(self):
        '''...'''
        s = super().__repr__()
        if self:
            return s.replace('})',f'}}, p={self.prime})')
        if not self:
            return s.replace('()', f'({{}}, p={self.prime})')
        
    def __str__(self):
        '''...'''
        s = super().__str__()
        s = s.replace(', ', ',')
        return s.replace('(','a^(')

#_________________________________79_characters________________________________

In [ ]:
## Reusable code

def nondegenerate(simplex):
    '''Returns True if the simplex is non-degenerate and False otherwise'''
    for i in range(len(simplex)-1):
        if simplex[i] == simplex[i+1]:
            return False
        
    return True

def clean(counter):
    '''Removes degenerated keys and reduces the coefficients of an element to 
    be 0,1,2 mod 3'''
    counter = Counter({key:value for (key,value) 
                       in counter.items() if nondegenerate(key)})
    counter = Counter({k: v%3 for k, v 
                       in counter.items()}) + Counter()
    return counter

def boundary(counter):
    bdry = Counter()
    for spx, coeff in counter.items():
        for i in range(len(spx)):
            bdry += Counter({tuple(spx[:i]+spx[i+1:])
                             : ((i%2+1)*coeff)%3})
    return clean(bdry)

def get_basis(n):
    '''Returns the preferred basis of $\mathcal E(\pi)_n$'''
    basis = [group_element for group_element 
             in product((0,1,2), repeat=n+1) 
             if nondegenerate(group_element)]
    
    return basis

def join(simplex, counter):
    '''Returns the Counter resulting from changing the 
    keys of counter via the join with simplex'''
    counter = Counter({simplex + elmt: coeff 
                       for elmt, coeff in counter.items()})
    
    return clean(counter)